In [1]:
import os
import sys
import wandb
import argparse
import numpy as np


sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "")))
import torch
import torchvision.transforms as T
import torchvision

from dataloaders.dataloader_cifar10 import get_cifar10
from dataloaders.dataloader_cifar100 import get_cifar100
from utils.eval_metrics import linear_evaluation
from models.linear_classifer import LinearClassifier
from models.ssl import  SimSiam, Siamese, Encoder, Predictor

import random
from utils.lr_schedulers import LinearWarmupCosineAnnealingLR, SimSiamScheduler
from utils.eval_metrics import Knn_Validation_cont
from copy import deepcopy
from loss import invariance_loss,CovarianceLoss,ErrorCovarianceLoss
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"

/home/duygu/anaconda3/envs/fedml_academic/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GaussianBlur(object):
    """Gaussian blur augmentation in SimCLR https://arxiv.org/abs/2002.05709"""

    def __init__(self, sigma=[0.1, 2.0]):
        self.sigma = sigma

    def __call__(self, x):
        sigma = random.uniform(self.sigma[0], self.sigma[1])
        x = torchvision.transforms.functional.gaussian_blur(x,kernel_size=[3,3],sigma=sigma)#kernel size and sigma are open problems but right now seems ok!
        return x


In [3]:
class Args():
    normalization = 'batch'
    weight_standard = False
    same_lr = True
    pretrain_batch_size = 512
    pretrain_warmup_epochs = 10
    pretrain_warmup_lr = 3e-3
    pretrain_base_lr = 0.03
    pretrain_momentum = 0.9
    pretrain_weight_decay = 5e-4
    min_lr = 0.00
    lambdap = 1.0
    appr = 'barlow_LRD'
    knn_report_freq = 10
    cuda_device = 1
    num_workers = 8
    contrastive_ratio = 0.001
    dataset = 'cifar100'
    class_split = [20,20,20,20,20]
    epochs = [500,500,500,500,500]
    cov_loss_weight = 1.0
    sim_loss_weight = 250.0
    info_loss = 'invariance'
    lambda_norm = 1.0
    subspace_rate = 0.99
    lambda_param = 5e-3
    bsize = 32
    msize = 150
    proj_hidden = 2048
    proj_out = 2048 #infomax 64
    pred_hidden = 512
    pred_out = 2048



In [4]:
args = Args()

In [5]:
if args.dataset == "cifar10":
    get_dataloaders = get_cifar10
    num_classes=10
elif args.dataset == "cifar100":
    get_dataloaders = get_cifar100
    num_classes=100
assert sum(args.class_split) == num_classes
assert len(args.class_split) == len(args.epochs)

In [6]:
num_worker = args.num_workers
#device
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


In [7]:
#wandb init
wandb.init(project="CSSL",  entity="yavuz-team",
            mode="disabled",
            config=args,
            name= str(args.dataset) + '-algo' + str(args.appr) + "-e" + str(args.epochs) + "-b" 
            + str(args.pretrain_batch_size) + "-lr" + str(args.pretrain_base_lr)+"-CS"+str(args.class_split))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [8]:
if 'infomax' in args.appr or 'barlow' in args.appr:
    transform = T.Compose([
            T.RandomResizedCrop(size=32, scale=(0.2, 1.0)),
            T.RandomHorizontalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)]), p=0.8),
            T.RandomGrayscale(p=0.2),
            T.RandomApply([GaussianBlur()], p=0.5), 
            T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])])

    transform_prime = T.Compose([
            T.RandomResizedCrop(size=32, scale=(0.2, 1.0)),
            T.RandomHorizontalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)]), p=0.8),
            T.RandomGrayscale(p=0.2),
            T.RandomApply([GaussianBlur()], p=0.5), 
            T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])])

In [9]:
#Dataloaders
print("Creating Dataloaders..")
#Class Based
train_data_loaders, train_data_loaders_knn, test_data_loaders, _, train_data_loaders_linear, train_data_loaders_pure  = get_dataloaders(transform, transform_prime, \
                                    classes=args.class_split, valid_rate = 0.00, batch_size=args.pretrain_batch_size, seed = 0, num_worker= num_worker)
_, train_data_loaders_knn_all, test_data_loaders_all, _, train_data_loaders_linear_all, train_data_loaders_pure_all = get_dataloaders(transform, transform_prime, \
                                        classes=[num_classes], valid_rate = 0.00, batch_size=args.pretrain_batch_size, seed = 0, num_worker= num_worker)


Creating Dataloaders..
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [10]:
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)
if 'infomax' in args.appr or 'barlow' in args.appr:
    proj_hidden = args.proj_hidden
    proj_out = args.proj_out
    encoder = Encoder(hidden_dim=proj_hidden, output_dim=proj_out, normalization = args.normalization, weight_standard = args.weight_standard,appr_name =args.appr)
    model0 = Siamese(encoder)
    model0.to(device) #automatically detects from model

cuda:1


In [11]:
model0.temporal_projector = nn.Sequential(
            nn.Linear(args.proj_out, args.proj_hidden, bias=False),
            nn.BatchNorm1d(args.proj_hidden),
            nn.ReLU(),
            nn.Linear(args.proj_hidden, args.proj_out),
        ).to(device)

In [12]:
#load model here
file_name = './checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_0_same_lr_True_norm_batch_ws_False.pth.tar'
dict = torch.load(file_name)

In [13]:
model0.load_state_dict(dict['state_dict'])

<All keys matched successfully>

In [14]:
#collect activations
def collect_activations(model,loader):
    activations = []

    model.eval()
    with torch.no_grad():
        for data_tuple in loader:
            data, target = [t.to(device) for t in data_tuple]
            # Forward prop of the model with single augmented batch
            output = model(data)
            activations.append(output.cpu().numpy())

    activations = np.concatenate(activations)
    return activations

def get_eigenspectrum(activations, max_eigvals=2048):
    """
    Given a (n x n) covariance matrix, compute the 
    eigenspectrum and the coefficient of decay by 
    fitting to the log-spectrum.
    """
    act_shape = activations.shape
    feats = activations.reshape(act_shape[0], -1)

    # batchsize x featdim
    bsz, feat_dim = feats.shape

    centered_ft = feats - feats.mean(axis=0)
    pca = PCA(n_components=min(max_eigvals, bsz, feat_dim), svd_solver='full')
    pca.fit(centered_ft)
    eigenspectrum = pca.explained_variance_ratio_
    return eigenspectrum

In [15]:
activations = collect_activations(model0, train_data_loaders_knn[0])

In [16]:
#load model here
file_name = './checkpoints/checkpoint_cifar100-algocassle_barlow-e[500, 500, 500, 500, 500]-b256-lr0.25-CS[20, 20, 20, 20, 20]_task_4_same_lr_True_norm_batch_ws_False.pth.tar'
#file_name = 'checkpoints/checkpoint_cifar100-algobasic_barlow-e[1000]-b256-lr0.3-CS[100]acc_69.65.pth.tar'
#file_name = 'checkpoints/checkpoint_cifar100-algoLRD_cross_barlow-e[500, 500, 500, 500, 500]-b256-lr0.1-CS[20, 20, 20, 20, 20]acc_57.29.pth.tar'
dict = torch.load(file_name)

In [17]:
device = torch.device("cuda:" + str(args.cuda_device) if torch.cuda.is_available() else "cpu")
print(device)
if 'infomax' in args.appr or 'barlow' in args.appr:
    proj_hidden = args.proj_hidden
    proj_out = args.proj_out
    encoder = Encoder(hidden_dim=proj_hidden, output_dim=proj_out, normalization = args.normalization, weight_standard = args.weight_standard,appr_name =args.appr)
    model1 = Siamese(encoder)
    model1.to(device) #automatically detects from model

cuda:1


In [18]:
model1.temporal_projector = nn.Sequential(
            nn.Linear(args.proj_out, args.proj_hidden, bias=False),
            nn.BatchNorm1d(args.proj_hidden),
            nn.ReLU(),
            nn.Linear(args.proj_hidden, args.proj_out),
        ).to(device)

In [19]:
model1.load_state_dict(dict['state_dict'])

<All keys matched successfully>

In [20]:
activations1 = collect_activations(model1, train_data_loaders_knn[0])

In [21]:
def extract_subspace(model, loader, rate=0.99,device = None, Q_prev = None, task=None):
    model.eval()
    outs = []
    for x,y in loader:
        x = x.to(device)
        out = model(x).cpu().detach().numpy()
        outs.append(out)

    outs = np.concatenate(outs)
    outs = outs.transpose()
    outs = torch.tensor(outs)

    if Q_prev == None:
        projected = torch.zeros(1)
    else:
        Q_prev = Q_prev.to('cpu')
        projected = Q_prev  @ Q_prev.T @ outs 

    remaining = outs - projected
    U, S, V = torch.svd(remaining)
    for i in range(len(S)):
        total = torch.norm(outs)**2 
        hand =  torch.norm(projected)**2 + torch.norm(S[0:i+1])**2
        
        if hand / total > rate:
            break

    print(U[:,0:i+1].shape)

    if Q_prev == None:
        Q_prev = U[:,0:i+1]
    else:
        Q_prev = torch.cat((Q_prev, U[:,0:i+1]),dim=1)
        Q_prev, _ = torch.linalg.qr(Q_prev, mode="reduced")
    print(Q_prev.shape)
    return Q_prev

In [22]:
Q = extract_subspace(model1,train_data_loaders_knn[0],device=args.cuda_device,rate=0.99)

torch.Size([512, 406])
torch.Size([512, 406])


In [23]:
activations_projected = torch.tensor(activations) @ Q @ Q.T 

In [24]:
torch.norm(activations_projected)/torch.norm(torch.tensor(activations))

tensor(0.9651)